In [1]:
import numpy as np
import pandas as pd

In [9]:
# Load comparison table created earlier
df = pd.read_csv("data/processed/if_lof_score_comparison.csv")
print(df.shape)
df.head()

(100000, 3)


,if_score,lof_score,isFraud
0,0.360532,0.977001,0
1,0.326089,0.984578,0
2,0.333120,0.985754,0
3,0.407142,1.029077,0
4,0.327719,0.985974,0


In [10]:
df["if_rank"] = df["if_score"].rank(method="first", ascending=False)
df["lof_rank"] = df["lof_score"].rank(method="first", ascending=False)

df.head()

,if_score,lof_score,isFraud,if_rank,lof_rank
0,0.360532,0.977001,0,28491.0,87745.0
1,0.326089,0.984578,0,68753.0,77026.0
2,0.333120,0.985754,0,55218.0,75210.0
3,0.407142,1.029077,0,13562.0,21246.0
4,0.327719,0.985974,0,65095.0,74882.0


In [11]:
df["avg_rank"] = (df["if_rank"] + df["lof_rank"]) / 2

Transactions ranked highly by either model move up

In [12]:
df["min_rank"] = df[["if_rank", "lof_rank"]].min(axis=1)

If any model thinks it’s risky → treat as risky.

In [13]:
df["weighted_rank"] = 0.6 * df["lof_rank"] + 0.4 * df["if_rank"]

From disagreement analysis:
	•	LOF stronger at tight K
	•	IF stronger deeper.So we weight LOF slightly higher

In [14]:
def precision_at_k(ranks, labels, k_pct):
    k = int(len(ranks) * k_pct / 100)
    top_k = ranks.nsmallest(k).index
    return labels.loc[top_k].mean()

In [15]:
for k in [0.1, 0.5, 1.0]:
    print(f"\nPrecision@{k}%")
    print("IF only:       ", precision_at_k(df["if_rank"], df["isFraud"], k))
    print("LOF only:      ", precision_at_k(df["lof_rank"], df["isFraud"], k))
    print("Avg rank:      ", precision_at_k(df["avg_rank"], df["isFraud"], k))
    print("Min rank:      ", precision_at_k(df["min_rank"], df["isFraud"], k))
    print("Weighted rank: ", precision_at_k(df["weighted_rank"], df["isFraud"], k))


Precision@0.1%
IF only:        0.0
LOF only:       0.21
Avg rank:       0.13
Min rank:       0.06
Weighted rank:  0.23

Precision@0.5%
IF only:        0.148
LOF only:       0.27
Avg rank:       0.352
Min rank:       0.164
Weighted rank:  0.362

Precision@1.0%
IF only:        0.249
LOF only:       0.235
Avg rank:       0.329
Min rank:       0.211
Weighted rank:  0.32


In [16]:
def fraud_capture(ranks, labels, k_pct):
    k = int(len(ranks) * k_pct / 100)
    return labels.loc[ranks.nsmallest(k).index].sum()

for k in [0.5]:
    print(f"\nFraud captured @ {k}%")
    print("IF:       ", fraud_capture(df["if_rank"], df["isFraud"], k))
    print("LOF:      ", fraud_capture(df["lof_rank"], df["isFraud"], k))
    print("Avg rank: ", fraud_capture(df["avg_rank"], df["isFraud"], k))
    print("Min rank: ", fraud_capture(df["min_rank"], df["isFraud"], k))


Fraud captured @ 0.5%
IF:        74
LOF:       135
Avg rank:  176
Min rank:  82


### Rank-Based Ensemble Results

Rank aggregation significantly improves fraud detection performance under fixed review capacity. At a 0.5% review threshold, the weighted rank ensemble achieves a precision of 36.2%, outperforming both Isolation Forest (14.8%) and LOF (27.0%), while capturing 176 fraud cases—substantially more than either individual model.

These results demonstrate that combining global and local anomaly rankings via rank-based ensembling effectively exploits complementary risk signals without relying on label supervision or score calibration.

In [19]:
for w in [0.5, 0.6, 0.7]:
    df["w_rank"] = w * df["lof_rank"] + (1 - w) * df["if_rank"]
    
    print(f"\nWeighted rank (LOF weight = {w})")
    for k in [0.1, 0.5, 1.0]:
        p = precision_at_k(df["w_rank"], df["isFraud"], k)
        print(f"  Precision@{k}%: {p:.3f}")


Weighted rank (LOF weight = 0.5)
  Precision@0.1%: 0.130
  Precision@0.5%: 0.352
  Precision@1.0%: 0.329

Weighted rank (LOF weight = 0.6)
  Precision@0.1%: 0.230
  Precision@0.5%: 0.362
  Precision@1.0%: 0.320

Weighted rank (LOF weight = 0.7)
  Precision@0.1%: 0.270
  Precision@0.5%: 0.354
  Precision@1.0%: 0.314


### Weight Sensitivity Analysis

Varying the relative contribution of LOF and Isolation Forest rankings results in smooth and predictable changes in Precision@K. Higher LOF weight improves performance under very tight review budgets, while balanced or IF-weighted rankings perform better at deeper review depths.

Overall, ensemble performance remains stable across reasonable weight choices, indicating that gains are driven by complementary ranking signals rather than precise parameter tuning.

In [17]:
df[[
    "if_rank",
    "lof_rank",
    "avg_rank",
    "min_rank",
    "weighted_rank",
    "isFraud"
]].to_csv("data/processed/rank_ensemble_scores.csv", index=False)
